# Red neuronal artificial en Numpy - MNIST

Bibliografía:
- Deep Learning with Python (capítulo 2), Francois Chollet, 2018 Manning
- https://www.coursera.org/learn/neural-networks-deep-learning/lecture/6dDj7/backpropagation-intuition-optional, Andrew Ng, 2018 DeepLearning.ai
- Learning Tensor Flow, Tom Hope, Yehezkel S. Resheff & Itay Lieder, 2017 O'Reilly
- TensorFlow for Deep Learning, Bharath Ramsundar & Reza Bosagh Zadeh, 2018, O'Reilly
- Python Machine Learning, 2nd ed. (capítulo 13), Sebastian Rachska, 2017 Packt
- Machine Learning with TensrFlow (capítulo 7), Nishant Shukla, 2018 Manning


Vamos en este notebook a implementar una red neuronal que permita la clasificación de imágenes de dígitos en escala de grises, utilizando tres niveles de extracción de programación.

Además de utilizar **numpy**, aprovechamos para presentar las librerías **tensorflow** y **keras** a través de su aplicación al problema de clasificación mencionado. 
El modelo a implementar es una red con una capa escondida de 512 neuronas utilizando la función de activación **ReLU** y 10 neuronas de salida utilizando la función de activación **softmax**.

Vamos a seguir un protocolo de *holdout* para evaluar los clasificadores.

# Parte 1. Exploración y entendimiento del dataset MNIST

## Librerías

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score

In [ ]:
import tensorflow as tf
tf.__version__

Keras necesita una base para trabajar. Vamos a usar tensorflow

In [ ]:
import keras
keras.__version__

Keras viene con MNIST directamente integrada. Las imagenes y sus etiquetas (clases) ya vienen particionadas en training y test sets.

## Carga de datos

In [ ]:
from keras.datasets import mnist
(train_images_ori, train_labels_ori), (test_images_ori, test_labels_ori) = mnist.load_data()

In [ ]:
train_images_ori[0][5] #La primera imagen, la 6a fila, muestra los 28 valores de gris (uno por cada columna)

In [ ]:
train_images_ori.shape

La primera dimensión del tensor de train_images es de 60000, pues hay 60000 imagenes, las 2 siguientes dimensiones son las que dan el tamaño de las imágenes (28x28). Como son imagenes en escala de grises, no hay una 4a dimensión que tendría el canal de color (e.g. RGB). Cada valor de la matriz de 28x28 es una valor de gris de 0 a 255.

In [ ]:
len (train_labels_ori)

In [ ]:
len (test_labels_ori)

El testset tiene 10000 imágenes

In [ ]:
train_labels_ori

In [ ]:
test_images_ori.shape

Podemos ver cada una de las imágenes

In [ ]:
digit = train_images_ori[4]
plt.imshow(digit, cmap=plt.cm.binary)
plt.show()

## Preprocesamiento de los datos

Necesitamos que los datos de entrenamiento y test estén en el formato dado por la capa de entrada de la red, donde se reciben los datos representados a partir de un tensor cuya primera dimensión tiene 784 neuronas, es decir, recibe como input instancias representadas por tensores con dimensionalidad (784,).

Cómo los datos de entrenamiento están en tensores de 60000x28x28, es necesario convertirlos en un tensor de 60000x784. Idem para el tensor de test.

Vamos también a modificar la escala de grises, que originalmente va de 0 a 255, a una escala que vaya de 0 a 1. Para esto, dividimos los valores originales por 255.

Definimos la estructura de la red (número de neuronas por capa y número de inputs).

In [ ]:
n0 = 28*28
n1 = 512
n2 = 10

Vamos a aplanar los datos de train y test para poder utilizar cada pixel como un predictor (cada pixel en una columna diferente).

In [ ]:
train_images = train_images_ori
train_images = train_images.reshape((60000, 28 * 28))
train_images = train_images.astype('float32') / 255

test_images = test_images_ori
test_images = test_images.reshape((10000, 28 * 28))
test_images = test_images.astype('float32') / 255

In [ ]:
train_images.shape

Las etiquetas están en valores numéricos, vamos codificarlos en one hot encoding, con un vector de K posiciones para K clases, donde la clase específica de cada ejemplo tiene un valor de 1, y las demas posiciones del vector tienen valores de 0

In [ ]:
train_labels_ori[0:10]

In [ ]:
from keras.utils import to_categorical

train_labels = to_categorical(train_labels_ori)
test_labels = to_categorical(test_labels_ori)

In [ ]:
train_labels.shape

In [ ]:
test_labels.shape

In [ ]:
train_labels[0:2]

Como los labels están desordenados y no responde a ninguna organización sistemática, los podemos dejar así. Si siguieran un orden definido, sería necesario barajarlos para que el orden no influyera en el aprendizaje.

In [ ]:
X_train = train_images.T
y_train = train_labels.T
m=X_train.shape[1]

# Parte 2. MNIST: ANN con TensorFlow de bajo nivel

## Definición del dataflow graph

Es importante aclarar que la tarea inicial de definir el dataflow graph no implica ningún procesamiento numérico, solo el establecimiento del orden de las operaciones y de cómo se conectan entre ellas (lazy evaluation).

### Placeholders

Lo primero que hacemos es indicar las características de los inputs y outputs y crear los **placeholders** correspondientes: son una especie de variables a las cuales se les asignarán valores en un momento futuro, el medio a través del cual se envían datos a los grafos de TensorFlow:
- los inputs son las imágenes, cada una caracterizada por las 784 columnas correspondientes a los pixeles.
- los outputs son los 10 indicadores de las variables de salida del modelo multinomial (one hot encoding).

Tenemos que los *shapes* tienen algunos de los rangos cuyos números de dimensiones no están determinados prealablemente, para estos especificamos entonces el valor de **None**. Este es el caso del rango dedicado al número de registros de entrenamiento que vamos a utilizar (o de predicción), que no se puede ni debe definir estáticamente.

In [ ]:
tf.reset_default_graph()

In [ ]:
X = tf.placeholder(shape=[None, n0], dtype=tf.float32, name="input_X")  
y = tf.placeholder(shape=[None, n2], dtype=tf.float32, name="labels_Y")

### Variables

Los parámetros de la red (pesos y sesgos) se definen en otra estructura llamada **Variable**, diferente a los **placeholders**. Las variables son manipuladas durante los cálculos, sus valores son modificados durante el entrenamiento.

Los pesos de las conexiones entre capas se inicializan aleatoriamente; los sesgos se pueden inicializar en zeros.
No olvidemos inicializar el generador aleatorio de números para poder reproducir los resultados

Vamos a utilizar el generador pseudo aleatorio de TensorFlow, por lo que debemos inicializarlo.

In [ ]:
RANDOM_SEED = 1234
tf.set_random_seed(RANDOM_SEED)

In [ ]:
w1 = tf.Variable(tf.random_normal((n0, n1), stddev=0.1), name="capa1_W")
w2 = tf.Variable(tf.random_normal((n1, n2), stddev=0.1), name="capa2_W")
b1 = tf.Variable(tf.zeros((n1)), dtype=tf.float32, name="capa1_b")
b2 = tf.Variable(tf.zeros((n2)), dtype=tf.float32, name="capa2_b")

### Operaciones de entrenamiento y predicción

Ahora podemos definir las operaciones de la etapa de **feed-forward**, calculando directamente los valores de las activaciones de las capas 1 y 2. La capa 1 va a tener la tangente hiperbólica como función de activación, y la capa 2.

Es importante aclarar que TensorFlow incluye la activación **SoftMax** directamente con la función de costo de **cross-entropy**, por lo que no es necesario especificar la operación en la capa 2.

In [ ]:
a0 = X
#a1 = tf.nn.relu(tf.matmul(a0, w1)+b1, name="capa1_activacion")
a1 = tf.nn.tanh(tf.matmul(a0, w1)+b1, name="capa1_activacion")
a2 = tf.matmul(a1, w2, name="capa2_activacion")+b2
y_pred = a2

La etapa del **back-propagation** consiste en definir la función de costo que se utiliza, en este caso cross-entropy, y el optimizador para minimizarla.
Vamos a minimizar la función de costo, que definimos como un nodo del grafo ya que es una operación entre tensores, utilizando el método de descenso de gradiente (un paso de GD es otro nodo en el grafo).

In [ ]:
lr = 0.5 #learning rate
cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits_v2(labels=y, logits=y_pred), name="costo")
paso_gd = tf.train.GradientDescentOptimizer(lr, name="GD_Step").minimize(cost, name="min_GD_Step")

Definimos otra tarea que sirve para predecir la categoría de un elemento dado:

In [ ]:
predict = tf.argmax(a2, axis=1)

## Ejecución del dataflow graph

Definimos la sesión de tensor flow donde vamos a computar el dataflow graph e inicializar las variables pre definidas.

In [ ]:
sess = tf.Session()
init = tf.global_variables_initializer()
sess.run(init)

Visualicemos el dataflow graph utilizando un servicio web de TensorBoard (código copiado de: https://github.com/tensorflow/tensorflow/blob/master/tensorflow/examples/tutorials/deepdream/deepdream.ipynb)

In [ ]:
from IPython.display import clear_output, Image, display, HTML

def strip_consts(graph_def, max_const_size=32):
    """Strip large constant values from graph_def."""
    strip_def = tf.GraphDef()
    for n0 in graph_def.node:
        n = strip_def.node.add() 
        n.MergeFrom(n0)
        if n.op == 'Const':
            tensor = n.attr['value'].tensor
            size = len(tensor.tensor_content)
            if size > max_const_size:
                tensor.tensor_content = "<stripped %d bytes>"%size
    return strip_def

def show_graph(graph_def, max_const_size=32):
    """Visualize TensorFlow graph."""
    if hasattr(graph_def, 'as_graph_def'):
        graph_def = graph_def.as_graph_def()
    strip_def = strip_consts(graph_def, max_const_size=max_const_size)
    code = """
        <script>
          function load() {{
            document.getElementById("{id}").pbtxt = {data};
          }}
        </script>
        <link rel="import" href="https://tensorboard.appspot.com/tf-graph-basic.build.html" onload=load()>
        <div style="height:600px">
          <tf-graph-basic id="{id}"></tf-graph-basic>
        </div>
    """.format(data=repr(str(strip_def)), id='graph'+str(np.random.rand()))

    iframe = """
        <iframe seamless style="width:1200px;height:620px;border:0" srcdoc="{}"></iframe>
    """.format(code.replace('"', '&quot;'))
    display(HTML(iframe))

In [ ]:
show_graph(tf.get_default_graph().as_graph_def())

Ahora corremos las iteraciones de aprendizaje de los parámetros de la red neuronal; cada época está definida en por un paso del gradient descent que definimos. Especificamos entonces los valores de los placeholders de **X y Y**.
Después de cada época evaluamos como nos está yendo en el training set y en el test set.

Como los labels están desordenados y no responde a ninguna organización sistemática, los podemos dejar así. Si siguieran un orden definido, sería necesario barajarlos para que el orden no influyera en el aprendizaje.

Definimos los hiperparámetros del modelo, utilizados para aprender los pesos de las capas (los verdaderos parámetros).

In [ ]:
epocas = 100 # Iteraciones de backpropagation

In [ ]:
X_train = train_images.T
y_train = train_labels.T
m=X_train.shape[1]

In [ ]:
X_test = test_images.T
y_test = test_labels.T

In [ ]:
epocas = 100 # Iteraciones de backpropagation

In [ ]:
for epoca in range(epocas):
    sess.run(paso_gd, feed_dict={X: X_train.T, y: y_train.T})
    y_train_pred = sess.run(predict, feed_dict={X: X_train.T, y: y_train.T})
    y_test_pred = sess.run(predict, feed_dict={X: X_test.T, y: y_test.T})
    
    y_train_real = np.argmax(y_train, axis=0)
    train_accuracy = np.mean(np.equal(y_train_real, y_train_pred))
    y_test_real = np.argmax(y_test, axis=0)
    test_accuracy = np.mean(np.equal(y_test_real, y_test_pred))
    test_accuracy
    
    if epoca%10 ==0:
        print("Epoca: %d, train accuracy = %.4f%%, test accuracy = %.4f%%"
              % (epoca, 100. * train_accuracy, 100. * test_accuracy))

print("Epoca: %d, train accuracy = %.4f%%, test accuracy = %.4f%%"
    % (epoca, 100. * train_accuracy, 100. * test_accuracy))
sess.close()

In [ ]:
print(confusion_matrix(y_test_pred, y_test_real))
print(classification_report(y_test_pred, y_test_real))

# Parte 3. Modelo utilizando el módulo *layers* de TensorFlow

Vamos ahora a utilizar el módulo **layers**, que permite abstraer las capas de tal manera que no es necesario definici sus operaciones internas.

Primero vamos a limpiar el grafo por defecto:

In [ ]:
tf.reset_default_graph()
RANDOM_SEED = 1234
tf.set_random_seed(RANDOM_SEED)

Definimos los placeholders. Vamos a ilustrar como se puede utilizar le método one_hot de TF para transformar de un array de valores con las categorías a uno codificado one hot dentro de TF.

In [ ]:
X = tf.placeholder(shape=(None, n0), dtype=tf.float32, name="Inputs_imagenes")  
y_una_cifra = tf.placeholder(shape=(None), dtype=tf.uint8, name="Labels")
y_onehot = tf.one_hot(indices=y_una_cifra, depth=n2, name="Labels_1hot")

Definimos las capas de la red, con sus funciones de activación

In [ ]:
h1 = tf.layers.dense(inputs=X, units=n1, activation=tf.nn.tanh, name="Capa_escondida")
#w1 = tf.Variable(tf.random_normal((n0, n1), stddev=0.1))
#b1 = tf.Variable(tf.zeros((n1)), dtype=tf.float32)
logits = tf.layers.dense(inputs=h1, units=n2, activation=None, name="Logits_pred")
#w2 = tf.Variable(tf.random_normal((n1, n2), stddev=0.1))
#b2 = tf.Variable(tf.zeros((n2)), dtype=tf.float32)

Definimos las operaciones de prediccion, tanto de clases como de probabilidades

In [ ]:
preds = {
    'clases': tf.argmax(logits, axis=1, name="pred_clases"),
    'probas': tf.nn.softmax(logits, name="softmax")
}

Definimos las funciones de incialización, costo, optimizador y paso de optimización

In [ ]:
init = tf.global_variables_initializer()
costo = tf.losses.softmax_cross_entropy(onehot_labels=y_onehot, logits=logits)
optimizador = tf.train.GradientDescentOptimizer(learning_rate=lr, name="GD_optim")
pasoGD = optimizador.minimize(loss = costo, name="pasoGD")

In [ ]:
show_graph(tf.get_default_graph().as_graph_def())

Ahora creamos el ciclo de entrenamiento de backpropagation.

In [ ]:
sess =tf.Session()
sess.run(init)

for epoca in range(epocas):
    costos = []
    feed = {X: train_images, y_una_cifra:train_labels_ori}
    _, costo_epoca, y_train_pred_class, y_train_pred_proba = sess.run(
        [pasoGD, costo, preds['clases'], preds['probas']], feed_dict=feed)
    train_accuracy = np.mean(np.equal(train_labels_ori, y_train_pred_class))

    feed = {X: test_images, y_una_cifra:test_labels_ori}
    y_test_pred_class, y_test_pred_proba = sess.run(
        [preds['clases'], preds['probas']], feed_dict=feed)
    test_accuracy = np.mean(np.equal(test_labels_ori, y_test_pred_class))

    if epoca%10 ==0:
        costos.append(costo_epoca)
        print("Epoca %2d: costo: %.6f, acc_train=%.6f, acc_test=%.6f" 
              % (epoca, np.mean(costos), train_accuracy, test_accuracy))

sess.close()

In [ ]:
print(confusion_matrix(y_test_pred_class, test_labels_ori))
print(classification_report(y_test_pred_class, test_labels_ori))

Por alguna razón, TensorFlow no cuenta con una implementación de mini-batch, pero nada nos impide implementarla (no lo haremos).

# Parte 4. MNIST: ANN con Keras Sequential

Keras es una capa de abstracción por encima de un framework de Deep Learning de backend.
Los backends actualmente soportados por Keras son: TensorFlow (Google), Theano (Université de Montréal), CNTK (Microsoft).
Gracias a este nivel de abstracción, se podría escribir un solo código y cambiar por completo de backend sin necesidad de modificar una línea de código.
Keras ha sido adoptado como parte de TensorFlow por Google.


In [ ]:
import keras
keras.__version__

Keras tiene dos tipos de modelo: 
- Secuencial: diseñado para arquitecturas de red senciallas, limitándos a procesamientos encadenados en un solo sentido
- Funcional: flexible, permite cualquier tipo de estructura general de procesamiento con o sin ciclos, con una o varias salidas

## Módelo secuencial de Keras

Definimos entonces el mismo modelo de una capa escondida que con los otros dos códigos:
- la capa de entrada no se especifica directamente, esta va a tener 28x28 = 784 nodos de entrada, 1 por cada pixel de las imágenes
- una capa escondida de 512 neuronas, que utiliza una función de activación **RELU**
- una capa de salida de 10 neuronas, una para cada clase, que utiliza una función de activación **softmax**, que permite obtener en la salida una distribución de probabilidad por cada una de las clases posibles.

In [ ]:
from keras import models
from keras import layers
from keras import optimizers

network = models.Sequential()
network.add(layers.Dense(n1, activation='tanh', input_shape=(n0,)))
network.add(layers.Dense(n2, activation='softmax'))

A medida que las capas son más profundas, las representaciones de los imputs van cambiando.
Cada capa se puede interpretar como un filtro, los datos entran con una representación y salen con una representación más útil para la tarea en cuestión (en este caso, la clasificación de imágenes).

Para que el modelo quede listo para su uso, hay que compilarlo y definir 3 cosas:
* una **función de pérdida**: que define como se va medir la calidad del modelo al aplicarlo a los datos de entrenamiento, entre más baja la pérdida, mejor la calidad de la tarea ejecutada.
* un **optimizador**: el mecanismo que utiliza la red para actualizar los pesos, basándose en la función de perdida.
* las **métricas**: a monitorear durante el entrenamiento y evaluación de la red

La función de perdida es la **categorical_crossentropy**, que se utiliza en el caso de problemas de clasificación multiclase. El optimizador tratará de minimizarla. Es una función de distancia entre el valor real (que solo tiene una de las dimensiones con un 1 y las demás con un 0) y el predicho por el modelo (que tiene una distribución que probablemente no sea absoluta, sino que reparta la unidad de probabilidad en cada una de las clases).
Esta función es la suma negativa de las entropias de cada una de las posiciones del vector one hot encoded de clases:
$$H(y,\tilde{y}) =-\sum{y*log(\tilde{y})}$$
Como el valor de *y* es solo 1 en una de las posiciondes del vector de clases y 0 en las demás, solo una posición será considerada, de tal manera que la similitud entre los valores reales (1) y predichos (fracción de 1) es dada por el log del valor predicho para la clase real.  

Vamos a utilizar un optimizador **SGD** (Stochastic Gradient Descent), pero sin sacarle provecho a su funcionalidad de modificaicón del learning rate, de tal manera que quede igual al gradient descent más simple.

Para hacerle seguimiento a la evolución de la calidad del modelo, utilizaremos la métrica de **accuracy**.

In [ ]:
lr = 0.5
epocas = 100

In [ ]:
sgd = optimizers.SGD(lr=lr, decay=0, momentum=0, nesterov=False)
network.compile(optimizer=sgd,
                loss='categorical_crossentropy',
                metrics=['accuracy'])

Finalmente, se tiene que aprender el modelo, llamando al método *fit* (lo que es diferente a compilarlo). Se tiene entonces que:
* definir los datos de entrada (train_images) y sus clases correspondientes (train_labels), cuyas dimensiones corresponden a las establecidas por la red que definimos.
* establecer el número de épocas o generaciones de entrenamiento, i.e. el número de veces que se va a pasar el dataset de entrenamiento por el modelo.
* establecer el tamaño de los paquetes de entrada a considerar antes de hacer cada actualización. Se podría definir por ejemplo en 128 imágenes. Es decir que para cada época que considera 60000 imágenes de entrenamiento, se tendrían 469 batch de entrenamiento de 128 imágenes cada uno, y cada batch se actualizarían los parámetros de la red, sin necesidad de acabar una época. Vamos a establecer un tamaño de batch de 60000, para que solo se actualicen los parámetros una vez se procesen todas las imágenes, y poder así comparar con los resultados de los modelos anteriores.


In [ ]:
network.fit(train_images, train_labels, epochs=epocas, batch_size=m)

Se obtuvo al final 91.15% de exactitud con el set de entrenamiento.

También podemos definir un batch size más pequeño, como lo hicimos con la red que creamos con numpy:

In [ ]:
network.fit(train_images, train_labels, epochs=epocas, batch_size=50)

Evaluemos ahora con el set de test.

In [ ]:
test_loss, test_acc = network.evaluate(test_images, test_labels)


In [ ]:
print('Test acc: ', test_acc)

Se obtuvo 98.18% de exactitud en el set de datos de evaluación, inferior al 98.88% del set de entrenamiento. Siempre hay que valuar con un dataset diferente al de entrenamiento para evitar sobreestimar la calidad del modelo.

Se puede mejorar utilizando métodos de regularización como las normas L1 o L2. En TF, las capas regularizadas incluyen el método en cuestión. Por ejemplo:

In [ ]:
from keras import regularizers
network.add(layers.Dense(n1, activation='tanh', input_shape=(n0,), kernel_regularizer=regularizers.l2(0.01)))

# Parte 5. Módelo funcional de Keras

La gran diferencia entre el API **Functional** de Keras con las del **Sequential**, es la flexibilidad:
- se pueden definir múltiples tensores de entrada y de salida
- se pueden tener capas utilizadas por varios modelos
- se pueden definir ciclos en las conexiones de las capas
- se pueden reutilizar modelos funcionales como si fueran capas para definir otros modelos que los utilizan

In [ ]:
from keras.layers import Input, Flatten, Dense#, Lambda
from keras.models import Model

### Creación del modelo

Se crea un tensor **Input** de entrada que define las dimensiones de entrada, que internamente va  crear un **placeholder** en TF.

In [ ]:
input_tensor = Input(shape = (784,))

Creamos ahora las capas densas, especificando el número de neuronas a utilizar y las funciones de activación correspondientes.

In [ ]:
hidden_output = Dense(n1, activation = 'tanh')(input_tensor)
classification_output = Dense(n2, activation = 'softmax')(hidden_output)

Ya tenemos todas las capas conectadas. Vamos ahora a crear el modelo que vamos a entrenar, definiendo el tensor donde quedarán los datos de entrada y el tensor donde quedarán los datos de salida.

In [ ]:
model = Model([input_tensor], [classification_output])
weights = model.get_weights()

In [ ]:
print(model.summary())

### Entrenamiento

Vamos ahora a entrenar el modelo, para lo que hay que definir el optimizador que vamos a utilizar, la función de pérdida que se va a minimizar y la métrica que se quiere monitorear:

In [ ]:
model.set_weights(weights)
model.compile(optimizer='rmsprop', loss='categorical_crossentropy', metrics=['accuracy'])
model.fit(train_images, train_labels)

In [ ]:
test_loss, test_acc = model.evaluate(test_images, test_labels)
print("test loss: {}, test accuracy: {}".format(test_loss, test_acc))

Con 1 época, el training set se llega a 91.8%, mientras que el test set produce 95.5% de accuracy. Vamos a intentar con un batch size de 50.

In [ ]:
tf.set_random_seed(1234)
model.compile(optimizer='rmsprop', loss='categorical_crossentropy', metrics=['accuracy'])
model.set_weights(weights)
history = model.fit(train_images, train_labels, epochs=10, batch_size=50, validation_split=0.2)  # starts training

In [ ]:
test_loss, test_acc = model.evaluate(test_images, test_labels)
print("test loss: {}, test accuracy: {}".format(test_loss, test_acc))

Con 10 épocas llegamos a 96.6% en el training set y 98% en el test set. 

El objeto **history** retornado por el entrenamiento (**fit**) permite tener una idea más clara de como se realizó el entrenamiento. Vamos a plotear la información que contiene.

In [ ]:
print(history.history.keys())

Podemos comparar los accuracy del training set y del test set.

In [ ]:
plt.plot(history.history['acc'])
plt.plot(history.history['val_acc'])
plt.title('model accuracy')
plt.ylabel('accuracy')
plt.xlabel('epoch')
plt.legend(['train', 'test'], loc='upper left')
plt.show()

Y lo propio con el loss

In [ ]:
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('model loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train', 'test'], loc='upper left')
plt.show()